In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_hiv12 = pd.read_stata("../Data/DataAll/12months.dta")

print(df_hiv12.info(verbose=False))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071 entries, 0 to 1070
Columns: 263 entries, patid to minidiag5
dtypes: datetime64[ns](6), float32(4), float64(246), object(7)
memory usage: 2.1+ MB
None


In [3]:
df_hiv12['heightst4'] /= 100  # Convert height to meters

# Calculate BMI and round to the nearest whole number
df_hiv12 = pd.concat([df_hiv12, (df_hiv12['weightst4'] / (df_hiv12['heightst4'] ** 2)).round().rename('BMI')], axis=1)

In [4]:
columns_to_convert = ['disrupt4', 'healthvisits4', 'hospdays4']

df_hiv12[columns_to_convert] = df_hiv12[columns_to_convert].apply(lambda x: (x > 0).astype(int))

df_hiv12['Psyc_index'] = df_hiv12[columns_to_convert].sum(axis=1, skipna=True)

C:\Users\Mutema\AppData\Local\Temp\ipykernel_29320\458901957.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_hiv12['Psyc_index'] = df_hiv12[columns_to_convert].sum(axis=1, skipna=True)


In [5]:
Soc_col = ['socneed4', 'socjoy4', 'socreal4', 'socemot4', 'soccomfo4', 'socfriend4',
           'socthing4', 'socprob4', 'socshare4', 'soclife4', 'socwill4', 'soctalk4']

df_hiv12['SS_score'] = pd.concat([df_hiv12[col] for col in Soc_col], axis=1).sum(axis=1) / len(Soc_col)

#print(df_hiv2['SS_score'].head())
# Create categorical column (SScat)
df_hiv12['SS_cat'] = pd.cut(df_hiv12['SS_score'], bins=[-float('inf'), 1, 3, 5.1, float('inf')],
                          labels=['NA', 'Low', 'Moderate', 'High'], include_lowest=True)
#Display the SS_cat
print(df_hiv12['SS_cat'].head())

0    Moderate
1    Moderate
2         Low
3    Moderate
4         Low
Name: SS_cat, dtype: category
Categories (4, object): ['NA' < 'Low' < 'Moderate' < 'High']


C:\Users\Mutema\AppData\Local\Temp\ipykernel_29320\4286021261.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_hiv12['SS_score'] = pd.concat([df_hiv12[col] for col in Soc_col], axis=1).sum(axis=1) / len(Soc_col)
C:\Users\Mutema\AppData\Local\Temp\ipykernel_29320\4286021261.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_hiv12['SS_cat'] = pd.cut(df_hiv12['SS_score'], bins=[-float('inf'), 1, 3, 5.1, float('inf')],


In [7]:
# Create list_a and list_b as subsets of df_hiv
list_a_columns = ['patid', 'father4', 'mother4', 'parentill4', 'parmisunder4', 'parneglect4', 'brother4',
                  'sister4', 'siblingill4', 'siblingmis4', 'siblingneg4', 'spouse4', 'lover4',
                  'spoloverill4', 'spolovmis4', 'spolovneg4', 'child4', 'childill4', 'childmis4',
                  'childcare4', 'ill4', 'anymis4', 'feelneg4', 'famprov4', 'jobloss4', 'discrim4',
                  'cope4', 'moneywor4']
list_b_columns = ['patid', 'fathst4', 'mothst4', 'parst4', 'parmisst4', 'parnegst4', 'brost4',
                  'sisst4', 'siblingst4', 'sibmisst4', 'sibnegst4', 'spousest4', 'loverst4',
                  'spolovillst4', 'spolovmisst4', 'spolovnegst4', 'childst4', 'childillst4',
                  'childmisst4', 'childcarest4', 'illst4', 'anymisrest4', 'feelnegst4', 'famprovst4',
                  'joblossst2', 'discrimst4', 'copest4', 'moneyworst4']

list_a = df_hiv12[list_a_columns].copy()
list_b = df_hiv12[list_b_columns].copy()

# Merge the two dataframes by 'patid'
df_ssi = pd.merge(list_a, list_b, on='patid', how='inner')
# Select columns to sum across efficiently
columns_to_sum = [col for col in df_ssi.columns if not col.endswith("st")]

# Count the number of 1s in each row for selected columns
row_sums = np.sum((df_ssi[columns_to_sum] == 1).values, axis=1)

# Add the result as a new column "SSI" to df_hiv using pd.concat
df_hiv12 = pd.concat([df_hiv12, pd.Series(row_sums, name='SSI')], axis=1)

# Display the resulting DataFrame
print(df_hiv12['SSI'].head())

0     2
1    10
2     7
3     0
4     0
Name: SSI, dtype: int32


In [ ]:
df_hiv12['SDS'] = df_hiv12[['dayslost', 'daysunproductive']].sum(axis=1, skipna=True)

df_hiv12['SDS_cat'] = pd.cut(df_hiv12['SDS'],
                               bins=[-float('inf'), 0, 6, 12, float('inf')],
                               labels=['None', 'Low', 'Moderate', 'High'],
                               include_lowest=True)

In [4]:
pip install mca


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for mca: filename=mca-1.0.3-py2.py3-none-any.whl size=6038 sha256=6a57482c774fc575d32b6c9b476c8615263accbeab7a2981991114eddfe5d58f
  Stored in directory: c:\users\mutema\appdata\local\pip\cache\wheels\98\34\01\a42e47f7ccc0a4f4283780349f8d347391bd0d8e0d83198b5d
Successfully built mca



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\Mutema\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
